# Document tagging

In [12]:
import spacy
import pprint
import time

import pandas as pd

from collections import defaultdict
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

from tqdm import tqdm
from langchain_openai import  AzureChatOpenAI, ChatOpenAI
from dotenv import load_dotenv
from tools.dataset import get_decicontas_tags_df
from tools.prompt import generate_few_shot_ner_prompts
from tools.schema import (
    NERDecisao
)

load_dotenv()

True

In [ ]:
nlp = spacy.load("pt_core_news_sm")

In [14]:
df_decicontas = get_decicontas_tags_df()

In [15]:
df_decicontas

,text,tags
0,DECIDEM os Conselheiros do Tribunal de Contas ...,[]
1,DECIDEM os Conselheiros do Tribunal de Contas ...,[]
2,DECIDEM os Conselheiros do Tribunal de Contas ...,[]
3,DECIDEM os Conselheiros do Tribunal de Contas ...,[]
4,DECIDEM os Conselheiros do Tribunal de Contas ...,[]
...,...,...
1420,"Vistos, relatados e discutidos estes autos de ...",[RESSARCIMENTO]
1421,"Vistos, relatados e discutidos estes autos de ...",[RESSARCIMENTO]
1422,"Vistos, relatados e discutidos estes autos da ...","[MULTA, RESSARCIMENTO]"
1423,"Vistos, relatados e discutidos estes autos do ...",[RESSARCIMENTO]


# Experiments

* Convert existing few-shot
* 

In [ ]:
from typing import List, Literal
from pydantic import BaseModel, Field
from langchain_openai import AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate

# ----- Schema da saída -----
AllowedTag = Literal["MULTA", "OBRIGACAO", "RESSARCIMENTO", "RECOMENDACAO"]

class DocTags(BaseModel):
    """Tags jurídicas presentes no documento (subconjunto do conjunto permitido)."""
    tags: List[AllowedTag] = Field(
        default_factory=list,
        description="Lista de tags únicas entre MULTA, OBRIGACAO, RESSARCIMENTO, RECOMENDACAO."
    )
    rationale: str = Field(
        description="Brevíssima justificativa por tag (uma ou duas frases), útil para auditoria.",
    )

ALLOWED = ["MULTA","OBRIGACAO","RESSARCIMENTO","RECOMENDACAO"]

# ----- Seus modelos -----
gpt41_nano = AzureChatOpenAI(deployment_name="gpt-4-1-nano",  model_name="gpt-4-1-nano", temperature=0)
gpt41_mini = AzureChatOpenAI(deployment_name="gpt-4-1-mini",  model_name="gpt-4-1-mini", temperature=0)
gpt41      = AzureChatOpenAI(deployment_name="gpt-4-1",       model_name="gpt-4-1",      temperature=0)
gpt4o      = AzureChatOpenAI(deployment_name="gpt-4o",         model_name="gpt-4o",       temperature=0)
gpt35      = AzureChatOpenAI(deployment_name="gpt-35",         model_name="gpt-35-turbo", temperature=0)
gpt4turbo  = AzureChatOpenAI(deployment_name="gpt-4-turbo",    model_name="gpt-4",        temperature=0)

# ----- Prompt único (com instruções claras, anti-alucinação e exemplos reduzidos) -----
prompt = ChatPromptTemplate.from_messages([
    ("system",
     "Você é um classificador jurídico. Tarefa: marcar um documento com zero ou mais tags "
     "a partir do conjunto permitido {allowed}. Responda SOMENTE com tags do conjunto. "
     "Se nada se aplica, retorne lista vazia.\n"
     "Critérios:\n"
     "- MULTA: penalidades pecuniárias (valor monetário, multa cominatória, % sobre vencimentos etc.)\n"
     "- OBRIGACAO: obrigações de fazer/não fazer, prazos, determinações, recomendações vinculantes\n"
     "- RESSARCIMENTO: devolução aos cofres públicos, dano ao erário, imputação de débito\n"
     "- RECOMENDACAO: recomendações (não vinculantes), boas práticas, ajustes futuros\n"
     "Saída deve seguir o schema."),
    # 2 exemplos curtos (few-shot)
    ("user",
     "TEXTO:\n'Aplicar multa de R$ 3.000,00 ao gestor e determinar que publique o RREO em 10 dias.'"),
    ("assistant",
     "{'tags': ['MULTA','OBRIGACAO'], 'rationale':'Multa explícita; determinação de fazer com prazo.'}"),
    ("user",
     "TEXTO:\n'Encaminhar recomendações para aprimorar o controle interno e ressarcir o valor ao erário.'"),
    ("assistant",
     "{'tags': ['RECOMENDACAO','RESSARCIMENTO'], 'rationale':'Há recomendações e ordem de devolver valores.'}"),
    ("user", "TEXTO:\n{doc}")
])

def build_chain(llm):
    # Força saída estruturada Pydantic
    return (prompt | llm.with_structured_output(DocTags, method="json_schema", include_raw=False))

# Dicionário de chains por modelo
CHAINS = {
    "gpt41_nano": build_chain(gpt41_nano),
    "gpt41_mini": build_chain(gpt41_mini),
    "gpt41":      build_chain(gpt41),
    "gpt4o":      build_chain(gpt4o),
    "gpt35":      build_chain(gpt35),
    "gpt4turbo":  build_chain(gpt4turbo),
}


In [ ]:
import pandas as pd
from tqdm import tqdm

def predict_tags_single(text: str, model_name: str = "gpt4turbo"):
    chain = CHAINS[model_name]
    out: DocTags = chain.invoke({"doc": text, "allowed": ALLOWED})
    # Garantias finais
    tags = sorted({t for t in out.tags if t in ALLOWED})
    return {"tags": tags, "rationale": out.rationale}

def predict_tags_batch(df_text: pd.DataFrame, text_col="text", model_name="gpt4turbo"):
    preds = []
    for txt in tqdm(df_text[text_col].tolist()):
        res = predict_tags_single(txt, model_name=model_name)
        preds.append(res["tags"])
    out = pd.DataFrame({"text": df_text[text_col], "tags_pred_"+model_name: preds})
    return out


In [ ]:
import pandas as pd
from tqdm import tqdm

def predict_tags_single(text: str, model_name: str = "gpt4turbo"):
    chain = CHAINS[model_name]
    out: DocTags = chain.invoke({"doc": text, "allowed": ALLOWED})
    # Garantias finais
    tags = sorted({t for t in out.tags if t in ALLOWED})
    return {"tags": tags, "rationale": out.rationale}

def predict_tags_batch(df_text: pd.DataFrame, text_col="text", model_name="gpt4turbo"):
    preds = []
    for txt in tqdm(df_text[text_col].tolist()):
        res = predict_tags_single(txt, model_name=model_name)
        preds.append(res["tags"])
    out = pd.DataFrame({"text": df_text[text_col], "tags_pred_"+model_name: preds})
    return out
